## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-14-22-52-04 +0000,nypost,NYC Mayor Adams blames Bondi Beach terror atta...,https://nypost.com/2025/12/14/us-news/nyc-mayo...
1,2025-12-14-22-50-51 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/12/14/world/...
2,2025-12-14-22-47-34 +0000,nyt,What We Know About the Brown University Shooting,https://www.nytimes.com/2025/12/13/us/brown-un...
3,2025-12-14-22-46-41 +0000,nyt,Live Updates: 15 Victims and a Gunman Dead Aft...,https://www.nytimes.com/live/2025/12/14/world/...
4,2025-12-14-22-46-39 +0000,nyt,What We Know About the Bondi Beach Shooting in...,https://www.nytimes.com/2025/12/14/world/austr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,bondi,27
12,brown,24
5,beach,23
14,shooting,18
13,university,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
84,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...,143
101,2025-12-14-16-36-41 +0000,nypost,"Famed Australian rabbi Eli Schlanger, who just...",https://nypost.com/2025/12/14/world-news/famed...,105
13,2025-12-14-22-25-01 +0000,wapo,Live updates: 15 killed in Bondi Beach shootin...,https://www.washingtonpost.com/world/2025/12/1...,105
8,2025-12-14-22-36-01 +0000,startribune,Father and son gunmen kill at least 15 people ...,https://www.startribune.com/police-say-2-were-...,99
95,2025-12-14-17-20-49 +0000,nypost,Mamdani condemns Bondi Beach attack as ‘vile a...,https://nypost.com/2025/12/14/us-news/mamdani-...,94


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
84,143,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...
101,55,2025-12-14-16-36-41 +0000,nypost,"Famed Australian rabbi Eli Schlanger, who just...",https://nypost.com/2025/12/14/world-news/famed...
112,37,2025-12-14-13-40-25 +0000,cbc,Live updates: Attack on Australia’s Jewish com...,https://www.cbc.ca/news/world/livestory/hanukk...
105,33,2025-12-14-15-50-19 +0000,nypost,Trump admits he’s not sure GOP will hold onto ...,https://nypost.com/2025/12/14/us-news/trump-ad...
146,28,2025-12-14-10-00-00 +0000,wsj,The falling out between Rep. Marjorie Taylor G...,https://www.wsj.com/politics/policy/marjorie-t...
114,27,2025-12-14-13-02-16 +0000,nypost,Incredible moment Bondi Beach hero disarms ter...,https://nypost.com/2025/12/14/world-news/incre...
52,24,2025-12-14-20-48-00 +0000,wsj,Police Detain Person of Interest in Deadly Bro...,https://www.wsj.com/us-news/police-detain-pers...
60,22,2025-12-14-20-08-22 +0000,nypost,Chinese gaming billionaire reportedly sires mo...,https://nypost.com/2025/12/14/us-news/xu-bo-ch...
94,21,2025-12-14-17-39-34 +0000,nypost,NYC airports facing 6-hour delays as massive s...,https://nypost.com/2025/12/14/us-news/nyc-airp...
56,20,2025-12-14-20-28-35 +0000,nyt,Zelensky Offers Compromise for New Round of Uk...,https://www.nytimes.com/2025/12/14/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
